In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import os
import requests
from PIL import Image
from io import BytesIO
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
df = pd.read_csv('data_최종.csv')
len(df)

1566

In [3]:
df_image_ok = df.loc[df['대표이미지'].notnull()] 
df_image_ok_ind = df['대표이미지'].loc[df['대표이미지'].notnull()].index.tolist()
df_image_not_ok = df.drop(df_image_ok_ind)

In [4]:
df_image_not_ok.loc[df_image_not_ok['대표이미지'].isna()]
df_image_not_ok['대표이미지'].isna().sum()

888

## Naver Crawling 통해 대표 이미지 url가져오기
- 주소 + 시설명 검색 안되면 시설명으로만 검색해서 crawling

In [1]:
# 네이버 길찾기에 키워드 넣어서 가져오기 
driver = webdriver.Chrome()

# output_dir = 'culture_images'
# os.makedirs(output_dir,exist_ok = True)


def search_and_click(query):
    # 네이버 검색창에서 query에 해당하는 주소로 들어가기
    url = "https://map.naver.com/v5/search"
    driver.get(url)
    time.sleep(2)  
    
    try:   
        # 검색창에 쿼리 입력
        search_box = driver.find_element(By.CSS_SELECTOR,'input.input_search')
        search_box.send_keys(query)
        search_box.send_keys(Keys.RETURN)
        time.sleep(2)  # 검색 결과 로딩 대기
        
         # iframe 로드 및 전환
        # iframe으로 감싸져 있는 html페이지일 경우 그 안에 소스들을 못가져옴 
        # iframe을 변환후 다음 작업 진행 
        iframe_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "entryIframe"))
    )
        driver.switch_to.frame(iframe_element)
        
    
        # 가져오고자 하는 이미지 클래스 
        image_element = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "img.K0PDV._div,img.K0PDV"))
        )
        image_url = [img.get_attribute('src') for img in image_element]
        
        if image_url:
            return image_url[0]
        else:
            return None
    
    except Exception as e:
        print(f"이미지를 찾을 수 없습니다")
        #print(f"이미지를 찾을 수 없습니다: {query} - {e}")
        return None


def search_image(row):
    shop_name = row['콘텐츠명']
    Id = row['중분류']
    address_column = '새주소[도로명 주소]'
    shop_column = '콘텐츠명'
    
    query = f"{row[address_column]} {row[shop_column]}"
    image_url = search_and_click(query)
    
    if not image_url:  # 첫 번째 시도에서 실패한 경우
        print(f"첫 번째 시도 실패: {query},  다시 검색")
        query = f"{row[shop_column]}"
        image_url = search_and_click(query)
        
    if image_url:
        df_image_not_ok.at[index, '대표이미지'] = image_url
        print(f"Updated image URL for {Id} {shop_name}")

for index, row in df_image_not_ok.iterrows():
    search_image(row)

print("Image URL crawling and updating completed.")
driver.quit()

In [56]:
df_image_not_ok['대표이미지'][:20]

0     https://search.pstatic.net/common/?autoRotate=...
1     https://simg.pstatic.net/static.map/v2/map/sta...
2     https://search.pstatic.net/common/?autoRotate=...
3     https://search.pstatic.net/common/?autoRotate=...
4     https://search.pstatic.net/common/?autoRotate=...
5     https://simg.pstatic.net/static.map/v2/map/sta...
6     https://search.pstatic.net/common/?autoRotate=...
7     https://search.pstatic.net/common/?autoRotate=...
8     https://search.pstatic.net/common/?autoRotate=...
9     https://search.pstatic.net/common/?autoRotate=...
10    https://search.pstatic.net/common/?autoRotate=...
11    https://search.pstatic.net/common/?autoRotate=...
12    https://search.pstatic.net/common/?autoRotate=...
13    https://yeyak.seoul.go.kr/cmsdata/web_upload/s...
14    https://search.pstatic.net/common/?autoRotate=...
15    https://search.pstatic.net/common/?autoRotate=...
16    https://search.pstatic.net/common/?autoRotate=...
17    https://search.pstatic.net/common/?autoRot

In [57]:
df_image_not_ok.loc[df_image_not_ok['대표이미지'].isna()][['새주소[도로명 주소]','콘텐츠명']]

,새주소[도로명 주소],콘텐츠명
88,서울특별시 마포구 서강로 137,롯데리아 신촌로터리
173,서울특별시 동작구 흑석로11길 4,엉터리서민갈비중앙대점
183,서울특별시 관악구 신원로 35,순두부와 청국장 그리고 찜
772,서울특별시 양천구 신정로11길 50-15,키즈팡


- 이미지 크롤링 못한거 채워줌

In [39]:
fill1 = df_image_not_ok['콘텐츠명'] == '삼청공원 유아숲체험원'
df_image_not_ok.loc[fill1,'대표이미지'] = df_image_not_ok.loc[fill1,'대표이미지'].fillna('https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20191110_64%2F1573348628971yXbbt_JPEG%2FgOpgXhcpit09TARcEmfde8O-.jpg')
fill2 = df_image_not_ok['콘텐츠명'] == '꿈키움 유아숲체험원'
df_image_not_ok.loc[fill2,'대표이미지'] = df_image_not_ok.loc[fill2,'대표이미지'].fillna('https://yeyak.seoul.go.kr/cmsdata/web_upload/svc/20240528/1716884484195OOSYT4BU33ATXB8HGKB3AQJWO.jpg')
fill3 = df_image_not_ok['콘텐츠명'] == '숭인공원 유아숲체험원'
df_image_not_ok.loc[fill3,'대표이미지'] = df_image_not_ok.loc[fill3,'대표이미지'].fillna('https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=http%3A%2F%2Fblogfiles.naver.net%2FMjAxNzA1MjRfNzQg%2FMDAxNDk1NjE1NDA1OTU5.DYZG7OhDqydZhghoHcl46cYruTwED0vXWtVmH-2Cblcg.tB60iNzk80bXsdU0_iaVWCRZZggzNTvZGLTnMHFPdfYg.JPEG.ii202ii%2F%25C1%25BE%25B7%25CE_%25C0%25AF%25BE%25C6%25BD%25A3_%252814%2529.jpg')
fill4 = df_image_not_ok['콘텐츠명'] == '문화예술공원 유아숲체험원'
df_image_not_ok.loc[fill4,'대표이미지'] = df_image_not_ok.loc[fill4,'대표이미지'].fillna('https://search.pstatic.net/common/?autoRotate=true&quality=95&type=w750&src=https%3A%2F%2Fapis.naver.com%2Fplace%2Fpanorama%2Fthumbnail%2F947862831%2F0%3Fwidth%3D800%26height%3D400%26msgpad%3D1720500085951%26md%3DIwLYZCdk2svhnp%252FOiwSj9Tffp7c%253D')
fill5 = df_image_not_ok['콘텐츠명'] == '영축산 유아숲체험원'
df_image_not_ok.loc[fill5,'대표이미지'] = df_image_not_ok.loc[fill5,'대표이미지'].fillna('https://search.pstatic.net/common/?autoRotate=true&quality=95&type=w750&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20170621_57%2F1498006436954jcHtq_JPEG%2F186571498376.jpg')
fill6 = df_image_not_ok['콘텐츠명'] == '북한산공원 유아숲체험원'
df_image_not_ok.loc[fill6,'대표이미지'] = df_image_not_ok.loc[fill6,'대표이미지'].fillna('https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyMjA2MDdfMTA4%2FMDAxNjU0NTY4MjUwNzQw.aVAiU_WdLz2ifqUITEsz2xRMIbmhkgie8KVVR3_ro7Mg.gL73RJ2xpsqw4G5CPCj5ODU1jPLrA1AOCOvGQSru-Fkg.JPEG.pyj0225%2F20220527%25A3%25DF155248.jpg&type=sc960_832')
fill7 = df_image_not_ok['콘텐츠명'] == '수락산 유아숲체험원'
df_image_not_ok.loc[fill7,'대표이미지'] = df_image_not_ok.loc[fill7,'대표이미지'].fillna('https://search.pstatic.net/common/?autoRotate=true&quality=95&type=w750&src=https%3A%2F%2Fapis.naver.com%2Fplace%2Fpanorama%2Fthumbnail%2F1066279992%2F0%3Fwidth%3D800%26height%3D400%26msgpad%3D1720500356304%26md%3DV20kZTwIWXwNABHP1ZVvXZW%252BuMg%253D')
fill8 = df_image_not_ok['콘텐츠명'] == '불암산 유아숲체험장'
df_image_not_ok.loc[fill8,'대표이미지'] = df_image_not_ok.loc[fill8,'대표이미지'].fillna('https://search.pstatic.net/common/?autoRotate=true&quality=95&type=w750&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20221127_133%2F1669535593817xGXLP_JPEG%2F20221127_094951.jpg')
fill9 = df_image_not_ok['콘텐츠명'] == '아차산 유아숲체험원'
df_image_not_ok.loc[fill9,'대표이미지'] = df_image_not_ok.loc[fill9,'대표이미지'].fillna('https://search.pstatic.net/common/?autoRotate=true&quality=95&type=w750&src=https%3A%2F%2Fapis.naver.com%2Fplace%2Fpanorama%2Fthumbnail%2F948542095%2F0%3Fwidth%3D800%26height%3D400%26msgpad%3D1720500877809%26md%3DntE9Q3uSFxFCRMX381RpQVrI5jU%253D')
fill10 = df_image_not_ok['콘텐츠명'] == '오동 유아숲체험원'
df_image_not_ok.loc[fill10,'대표이미지'] = df_image_not_ok.loc[fill10,'대표이미지'].fillna('https://search.pstatic.net/common/?autoRotate=true&quality=95&type=w750&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20161214_97%2F1481690095287ejrhJ_JPEG%2F_1.jpg')
fill11 = df_image_not_ok['콘텐츠명'] == '서울숲공원 유아숲체험원'
df_image_not_ok.loc[fill11,'대표이미지'] = df_image_not_ok.loc[fill11,'대표이미지'].fillna('https://simg.pstatic.net/static.map/v2/map/staticmap.bin?center=127.038897%2C37.5493539&level=16&dataversion=172.83&caller=naver_mstore&markers=pos%3A127.038897%2037.5493539%7Ctype%3Ac%7Ccolor%3Ablue%7Clabel%3Arestaurant&w=400&h=200')
fill12 = df_image_not_ok['콘텐츠명'] == '봉화산 유아숲체험원'
df_image_not_ok.loc[fill12,'대표이미지'] = df_image_not_ok.loc[fill12,'대표이미지'].fillna('https://search.pstatic.net/common/?autoRotate=true&quality=95&type=w750&src=https%3A%2F%2Fapis.naver.com%2Fplace%2Fpanorama%2Fthumbnail%2F1082908676%2F0%3Fwidth%3D800%26height%3D400%26msgpad%3D1720501040620%26md%3Dysmpbw5kIPW7KgVpq1uesfyGziE%253D')
fill13 = df_image_not_ok['콘텐츠명'] == '응봉공원 유아숲체험원'
df_image_not_ok.loc[fill13,'대표이미지'] = df_image_not_ok.loc[fill13,'대표이미지'].fillna('https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20211214_75%2F1639471889296IBMEa_PNG%2Fnoname02.png')
fill14 = df_image_not_ok['콘텐츠명'] == '갈산 유아숲체험원'
df_image_not_ok.loc[fill14,'대표이미지'] = df_image_not_ok.loc[fill14,'대표이미지'].fillna('https://cafeptthumb-phinf.pstatic.net/MjAyMTAyMjdfMjY0/MDAxNjE0NDEwNzgyNzIy.rUuFLYONp1JhslzqobWTY-VXyso39IAZsZko_UuOOtcg.B2j5KgF-XJustbzVyZ8SVla1XxMyDYKwnTYenQU6ENMg.JPEG/20210227%EF%BC%BF160130.jpg?type=w1600')
fill15 = df_image_not_ok['콘텐츠명'] == '잣절 유아숲체험원'
df_image_not_ok.loc[fill15,'대표이미지'] = df_image_not_ok.loc[fill15,'대표이미지'].fillna('https://search.pstatic.net/common/?autoRotate=true&quality=95&type=w750&src=https%3A%2F%2Fapis.naver.com%2Fplace%2Fpanorama%2Fthumbnail%2F1242646060%2F0%3Fwidth%3D800%26height%3D400%26msgpad%3D1720501270112%26md%3DOtsyPm6Lk9kqJZ2Oup9FluI99ro%253D')
fill16 = df_image_not_ok['콘텐츠명'] == '인왕산 유아숲체험원'
df_image_not_ok.loc[fill16,'대표이미지'] = df_image_not_ok.loc[fill16,'대표이미지'].fillna('https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=http%3A%2F%2Fblogfiles.naver.net%2F20160607_232%2Fdufenwkqhs_1465253039064oUK7q_JPEG%2F20160606_111848.jpg')
fill17 = df_image_not_ok['콘텐츠명'] == '참나무본가'
df_image_not_ok.loc[fill17,'대표이미지'] = df_image_not_ok.loc[fill17,'대표이미지'].fillna('https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20231128_228%2F1701175860708Ntq4x_JPEG%2FDSC_0377.JPG')
fill18 = df_image_not_ok['콘텐츠명'] == 'KKD 메세나폴리스점'
df_image_not_ok.loc[fill18,'대표이미지'] = df_image_not_ok.loc[fill18,'대표이미지'].fillna('https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220826_115%2F166149031744176Q6B_JPEG%2F%25B3%25D7%25C0%25CC%25B9%25F6_%25B4%25EB%25C7%25A5%25C0%25CC%25B9%25CC%25C1%25F6_1_%25B4%25EB%25C1%25F6_1_%25B4%25EB%25C1%25F6_1.jpg')
fill19 = df_image_not_ok['콘텐츠명'] == '롯데리아 화곡점'
df_image_not_ok.loc[fill19,'대표이미지'] = df_image_not_ok.loc[fill19,'대표이미지'].fillna('https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20240207_300%2F1707268553325GfUKX_PNG%2F%25BA%25F1%25BA%25F6.png')
fill20 = df_image_not_ok['콘텐츠명'] == '엔제리너스 수서화승통로'
df_image_not_ok.loc[fill20,'대표이미지'] = df_image_not_ok.loc[fill20,'대표이미지'].fillna('https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220905_13%2F16623678242693JsIs_JPEG%2F%25BF%25A3%25C1%25A9%25B7%25CE%25B0%25ED1.jpg')
fill21 = df_image_not_ok['콘텐츠명'] == '여백집'
df_image_not_ok.loc[fill21,'대표이미지'] = df_image_not_ok.loc[fill21,'대표이미지'].fillna('https://search.pstatic.net/common/?autoRotate=true&quality=95&type=w750&src=https%3A%2F%2Fapis.naver.com%2Fplace%2Fpanorama%2Fthumbnail%2F37263605%2F0%3Fwidth%3D800%26height%3D400%26msgpad%3D1720503361820%26md%3Dk%252FZLVIRitemmpfT5fq0LbH7S3tA%253D')
fill22 = df_image_not_ok['콘텐츠명'] == '이디야커피 송파한양점'
df_image_not_ok.loc[fill22,'대표이미지'] = df_image_not_ok.loc[fill22,'대표이미지'].fillna('https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220905_13%2F16623678242693JsIs_JPEG%2F%25BF%25A3%25C1%25A9%25B7%25CE%25B0%25ED1.jpg')
fill23 = df_image_not_ok['콘텐츠명'] == '엔제리너스 강남역사거리'
df_image_not_ok.loc[fill23,'대표이미지'] = df_image_not_ok.loc[fill23,'대표이미지'].fillna('https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220905_13%2F16623678242693JsIs_JPEG%2F%25BF%25A3%25C1%25A9%25B7%25CE%25B0%25ED1.jpg')
fill24 = df_image_not_ok['콘텐츠명'] == '최고갈비 신정네거리점'
df_image_not_ok.loc[fill24,'대표이미지'] = df_image_not_ok.loc[fill24,'대표이미지'].fillna('https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230624_237%2F16875687698689EcQT_JPEG%2FKakaoTalk_20230624_100257907_02.jpg')
fill25 = df_image_not_ok['콘텐츠명'] == '능이버섯백숙'
df_image_not_ok.loc[fill25,'대표이미지'] = df_image_not_ok.loc[fill25,'대표이미지'].fillna('https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20150831_80%2F1440988547507IDMRP_JPEG%2F11869184_0.jpg')
fill26 = df_image_not_ok['콘텐츠명'] == '진국밥'
df_image_not_ok.loc[fill26,'대표이미지'] = df_image_not_ok.loc[fill26,'대표이미지'].fillna('https://search.pstatic.net/common/?autoRotate=true&quality=95&type=w750&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20200419_121%2F1587273541081LnGAm_JPEG%2FbJQtZk40haGUYnT8XJBHqpmH.jpeg.jpg')
fill27 = df_image_not_ok['콘텐츠명'] == '고덕명가굽은다리'
df_image_not_ok.loc[fill27,'대표이미지'] = df_image_not_ok.loc[fill27,'대표이미지'].fillna('https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20211217_42%2F1639730321611ONiWH_JPEG%2F1639730309209.jpg')
fill28 = df_image_not_ok['콘텐츠명'] == '이바돔감자탕 강동역점'
df_image_not_ok.loc[fill28,'대표이미지'] = df_image_not_ok.loc[fill28,'대표이미지'].fillna('https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20190323_288%2F1553310128159jkkMI_JPEG%2F7J62zZ_SoatQ7F5KShcB4QSV.jpg')
fill29 = df_image_not_ok['콘텐츠명'] == '엔제리너스 영등포점'
df_image_not_ok.loc[fill29,'대표이미지'] = df_image_not_ok.loc[fill29,'대표이미지'].fillna('https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220905_13%2F16623678242693JsIs_JPEG%2F%25BF%25A3%25C1%25A9%25B7%25CE%25B0%25ED1.jpg')
fill30 = df_image_not_ok['콘텐츠명'] == '황금터'
df_image_not_ok.loc[fill30,'대표이미지'] = df_image_not_ok.loc[fill30,'대표이미지'].fillna('https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20201230_285%2F1609318768167s1vY9_JPEG%2FQFkgU3rfltF6pa1Yd0CRTi1O.jpg')
fill31 = df_image_not_ok['콘텐츠명'] == '대가이바지 감자탕'
df_image_not_ok.loc[fill31,'대표이미지'] = df_image_not_ok.loc[fill31,'대표이미지'].fillna('https://postfiles.pstatic.net/MjAyMzExMjFfMjE3/MDAxNzAwNTMyMTUzMDU3.3wRwLnw_C2YXdAIIc3iupHOmm8dQZPmXxmpXv4hllI8g.zV5FY452YbDYfhD6O1YjGriOHmhARStPJC24VMjrrmcg.JPEG.wlgml2467/SE-f3bb2c87-d8fc-425e-8f9a-48abe1a9d448.jpg?type=w966')
fill32 = df_image_not_ok['콘텐츠명'] == '마포갈비집'
df_image_not_ok.loc[fill32,'대표이미지'] = df_image_not_ok.loc[fill32,'대표이미지'].fillna('https://postfiles.pstatic.net/20121112_291/lunaticrrr_13526521663004PBrz_JPEG/IMG_8880.JPG?type=w1')
fill33 = df_image_not_ok['콘텐츠명'] == '능이오리백숙'
df_image_not_ok.loc[fill33,'대표이미지'] = df_image_not_ok.loc[fill33,'대표이미지'].fillna('https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20150901_113%2F1441079763677q0hdN_JPEG%2F156060554657159_0.jpeg')
fill34 = df_image_not_ok['콘텐츠명'] == '건대스타시티'
df_image_not_ok.loc[fill34,'대표이미지'] = df_image_not_ok.loc[fill34,'대표이미지'].fillna('https://search.pstatic.net/common/?autoRotate=true&quality=95&type=w750&src=https%3A%2F%2Fapis.naver.com%2Fplace%2Fpanorama%2Fthumbnail%2F1041320352%2F0%3Fwidth%3D800%26height%3D400%26msgpad%3D1720541278296%26md%3D%252BiWCv%252FJHcbqGE6kDx%252Fjt%252BWUicvI%253D')
fill35 = df_image_not_ok['콘텐츠명'] == '키즈카페'
df_image_not_ok.loc[fill35,'대표이미지'] = df_image_not_ok.loc[fill35,'대표이미지'].fillna('https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20200604_63%2F159126266148148e9H_JPEG%2F%25BD%25E6%25B3%25D7%25C0%25CF%2528%25C1%25A4%25BB%25E8%25B0%25A2%25C7%25FC%2529.jpg')
fill36 = df_image_not_ok['콘텐츠명'] == '서울형 키즈카페 서초구 방배2동점'
df_image_not_ok.loc[fill36,'대표이미지'] = df_image_not_ok.loc[fill36,'대표이미지'].fillna('https://search.pstatic.net/common/?autoRotate=true&quality=95&type=w750&src=https%3A%2F%2Fapis.naver.com%2Fplace%2Fpanorama%2Fthumbnail%2F1535586045%2F0%3Fwidth%3D800%26height%3D400%26msgpad%3D1720541960570%26md%3Dw8B2nmlRYyEMC%252FTnTWdsvbRbRFE%253D')


In [ ]:
"""
서울특별시 강동구 고덕로 421	참나무본가 - 주소 잘못된듯
서울특별시 마포구 서강로 137	롯데리아 신촌로터리 - 폐점 
서울특별시 강남구 광평로51길 27	엔제리너스 수서화승통로 - 수서환승통로점임 
서울특별시 동작구 흑석로11길 4	엉터리서민갈비중앙대점 - 폐점 
서울특별시 관악구 신원로 35	순두부와 청국장 그리고 찜 - 폐점 
서울특별시 양천구 신정로11길 50-15	키즈팡 - 폐점
"""

In [59]:
df_image_not_ok.drop(columns='Unnamed: 0',inplace=True)

In [61]:
df_image_not_ok

콘텐츠명                   0
구명                     0
새주소[도로명 주소]           44
지번주소                  31
좌표[X]                  0
좌표[Y]                  0
전화번호                 133
주요시설                 860
중분류                    0
대분류                    0
실내_실외                  0
시군                   286
키워드                  292
웹URL                 640
키즈메뉴(아이가먹을수있는 메뉴)    319
유아용품                 318
영업시간                 320
주차가능여부               286
무료구분                 857
운영일                  857
휴관일                  857
이용정원(기관)             857
이용정원(개인)             857
신청가능연령               857
대표이미지                  4
시설소개                 888
카테고리_키워드               0
전체_키워드                 0
query                  0
kakao_review           0
kakao_review_cnt       0
naver_query            0
naver_review         888
naver_review_cnt     888
dtype: int64

In [62]:
finish_image = pd.concat([df_image_not_ok,df_image_ok])
len(finish_image)

1566

In [66]:
finish_image.to_excel('data_image_ok.xlsx',index=False)